In [2]:
#htmlを解析しデータを抽出するためのライブラリ
from bs4 import BeautifulSoup

#httpリクエストを行うためのライブラリ
#webページの情報を取得する
import requests

In [4]:
import requests
from bs4 import BeautifulSoup

# 特定のURL
url_main = 'https://kakomonn.com/keamane'

# requestsを使ってURLからHTMLコンテンツを取得
response = requests.get(url_main)


In [6]:
def get_url_text():
    # 取得したHTMLコンテンツ
    html_content = response.text

    # BeautifulSoupを使ってHTMLを解析
    soup = BeautifulSoup(html_content, 'html.parser')

    return soup

https://kakomonn.com
/itpass
/anma
/kansekou1
/1kenchikushi
/kenchikusekou1
/denkisekou1
/1dobokusekou
/iryoujimu
/kamotsu
/eigo
/fp2
/fp3
/kaigofukushi
/kashikin
/kanji
/nurse
/kanrieiyoushi
/kanrigyoumu
/otsu4
/gijyutushi
/kyuukou
/gyouseishoshi
/CraneDerrick
/keamane
/shinrishi
/kyosai
/kokunairyokou
/shihoushoshi
/shakaifukushi
/sharoushi
/seishinhoken
/sokuryoshiho
/1eiseikanrisha
/1denkikoujishi
/sekaishi
/denken3
/2eiseikanrisha
/2shudenkikoujishi
/takken
/chushoks
/chozai
/chourishi
/chintaikanrishi
/tsukanshi
/tourokuhanbaisha
/TOEIC
/kansekou2
/2kenchikushi
/kenchikusekou2
/denkisekou2
/2dobokusekou
/boiler2
/hisho
/biyoushi
/birukan
/futsumenkyo
/hoikushi
/mansionkanrishi
/yakuzaishi
/riyoushi
https://docs.google.com/forms/d/e/1FAIpQLSd5ADIsj5rbvOwS0kGi_MEDubegD2sJbOHsNHJEIiwUcdJZzQ/viewform?entry.1798133137=PC_Other&entry.2129208253=https://kakomonn.com/keamane
https://kakomonn.com/account/login
https://kakomonn.com/keamane
https://kakomonn.com/keamane/createques
https://ka

In [8]:
#配列を宣言
links = []

# 特定の条件にマッチするすべてのリンクを検索し、表示
for link in soup.find_all('a', href=True):
    href = link['href']
    # 'https://kakomonn.com/keamane/list1/'で始まるURLに限定
    if href.startswith('https://kakomonn.com/keamane/list1/'):
        links.append(href)


In [9]:
for link in links:
    print(link)

html_conten = response.text

https://kakomonn.com/keamane/list1/8019
https://kakomonn.com/keamane/list1/8018
https://kakomonn.com/keamane/list1/8017
https://kakomonn.com/keamane/list1/8016
https://kakomonn.com/keamane/list1/8015
https://kakomonn.com/keamane/list1/8014
https://kakomonn.com/keamane/list1/8013
https://kakomonn.com/keamane/list1/8012
https://kakomonn.com/keamane/list1/8011
https://kakomonn.com/keamane/list1/8010
https://kakomonn.com/keamane/list1/8009
https://kakomonn.com/keamane/list1/8008
https://kakomonn.com/keamane/list1/8007
https://kakomonn.com/keamane/list1/8006
https://kakomonn.com/keamane/list1/8005


In [11]:
# requestsを使ってURLからHTMLコンテンツを取得
response = requests.get(links[0])

'https://kakomonn.com/keamane/list1/8018'

In [ ]:
# BeautifulSoupを使ってHTMLを解析
soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
del df_all

In [19]:
del Aptitude_data_list

NameError: name 'Aptitude_data_list' is not defined

In [11]:
%%time

num = 2000
# スクレイピング結果を格納するリスト
Aptitude_data_list = []

# 血統番号で馬番号をループする
for id in blood_id[5361:len(blood_id)+1]:
    #len(blood_id+1)

    # URLの設定
    url = "https://db.netkeiba.com/horse/" + id + "/"

    # HTTPリクエストを行うための準備
    response = requests.get(url)

    # エンコーディング設定
    response.encoding = 'euc-jp'

    # HTMLの作成
    html = response.text

    # BeautifulSoupパーサーの作成
    soup = BeautifulSoup(html, "html.parser")

    # 探したい`<th>`タグのテキストのリスト
    search_texts = set(['コース適性', '距離適性', '脚質', '成長', '重馬場'])

    # スクレイピング結果を格納するリスト
    results = []

    # HTML属性の解析および値の抽出
    # `<dl class="tekisei">`を見つける
    tekisei_dl = soup.find('dl', class_='tekisei')

    if tekisei_dl:
        # `<dl>`内の`<tr>`タグをループ処理
        for tr in tekisei_dl.find_all('tr'):

            th = tr.find('th')

            if th and th.text in search_texts:

                # `<tr>`タグ内の`<td>`タグから`<img>`タグを探す
                width_attrs = [img['width'] for img in tr.find_all('img', width=True)]

                # 二番目の`width`属性の値を取得して保存
                if len(width_attrs) >= 2:
                    results.append(width_attrs[1])

    Aptitude_data_list.append({id: results})
    
    time.sleep(0.3)

    if len(Aptitude_data_list) == num:
        print(len(Aptitude_data_list))
        # 辞書型をpickle形式で保存
        with open(f'tekisei{num}.pickle', 'wb') as f:
            pickle.dump(Aptitude_data_list, f)
        num += 2000
        time.sleep(10.0)

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
CPU times: total: 1h 36min 16s
Wall time: 18h 44min 14s


In [12]:
len(blood_id)

88943

In [18]:
%%time

num = len(blood_id)
# スクレイピング結果を格納するリスト
Aptitude_data_list= []

# 血統番号で馬番号をループする
for id in blood_id[87361:len(blood_id)+1]:
    #len(blood_id+1)

    # URLの設定
    url = "https://db.netkeiba.com/horse/" + id + "/"

    # HTTPリクエストを行うための準備
    response = requests.get(url)

    # エンコーディング設定
    response.encoding = 'euc-jp'

    # HTMLの作成
    html = response.text

    # BeautifulSoupパーサーの作成
    soup = BeautifulSoup(html, "html.parser")

    # 探したい`<th>`タグのテキストのリスト
    search_texts = set(['コース適性', '距離適性', '脚質', '成長', '重馬場'])

    # スクレイピング結果を格納するリスト
    results = []

    # HTML属性の解析および値の抽出
    # `<dl class="tekisei">`を見つける
    tekisei_dl = soup.find('dl', class_='tekisei')

    if tekisei_dl:
        # `<dl>`内の`<tr>`タグをループ処理
        for tr in tekisei_dl.find_all('tr'):

            th = tr.find('th')

            if th and th.text in search_texts:

                # `<tr>`タグ内の`<td>`タグから`<img>`タグを探す
                width_attrs = [img['width'] for img in tr.find_all('img', width=True)]

                # 二番目の`width`属性の値を取得して保存
                if len(width_attrs) >= 2:
                    results.append(width_attrs[1])

    Aptitude_data_list.append({id: results})
    
    time.sleep(0.3)

# 辞書型をpickle形式で保存
with open(f'tekisei_Last.pickle', 'wb') as f:
    pickle.dump(Aptitude_data_list, f)

  

CPU times: total: 1min 18s
Wall time: 19min 4s


In [17]:
Aptitude_data_list

[{'2099104764': []},
 {'2099104768': []},
 {'2099104769': []},
 {'2099104770': []},
 {'2099104771': []},
 {'2099104772': []},
 {'2099104773': []},
 {'2099104775': []},
 {'2099104779': []},
 {'2099104782': []},
 {'2099104787': []},
 {'2099104789': []},
 {'2099104793': []},
 {'2099104796': []},
 {'2099104797': []},
 {'2099104798': []},
 {'2099104800': []},
 {'2099104805': []},
 {'2099104813': []},
 {'2099104816': []},
 {'2099104817': []},
 {'2099104818': []},
 {'2099104819': []},
 {'2099104820': []},
 {'2099104825': []},
 {'2099104828': []},
 {'2099104831': []},
 {'2099104832': []},
 {'2099104833': []},
 {'2099104837': []},
 {'2099104838': []},
 {'2099104840': []},
 {'2099104841': []},
 {'2099104842': []},
 {'2099104843': []},
 {'2099104847': []},
 {'2099104848': []},
 {'2099104849': []},
 {'2099104850': []},
 {'2099104851': []},
 {'2099104853': []},
 {'2099104854': []},
 {'2099104861': []},
 {'2099104863': []},
 {'2099104865': []},
 {'2099104867': []},
 {'2099104880': []},
 {'2099104882